## SAR data used

### Earthquakes Involved

+ 2023-02-06 01:17:34 (UTC) M 7.8 earthquake ([USGS](https://earthquake.usgs.gov/earthquakes/eventpage/us6000jllz/executive))
+ 2023-02-06 10:24:48 (UTC) M 7.5 earthquake ([USGS](https://earthquake.usgs.gov/earthquakes/eventpage/us6000jlqa/executive))
+ 2023-02-20 17:04:29 (UTC) M 6.3 earthquake ([USGS](https://earthquake.usgs.gov/earthquakes/eventpage/us6000jqcn/executive))

### ALOS-2

+ Analysis by GSI: https://www.gsi.go.jp/cais/topic20230206-e_Turkey.html
+ Data Info from JAXA: https://www.eorc.jaxa.jp/ALOS/en/dataset/alos_open_and_free_e.htm
+ Data Search and Download: https://gportal.jaxa.jp/gpr/
+ ARIA from JPL-Caltech: https://aria-share.jpl.nasa.gov/20230206_Turkey_EQ/Displacements/ALOS2

### Sentinel-1

+ ARIA Interferograms:
  - https://grfn.asf.alaska.edu/door/download/S1-GUNW-D-R-021-tops-20230210_20230129-033440-00036E_00037N-PP-c92c-v2_0_6.nc
  - https://grfn.asf.alaska.edu/door/download/S1-GUNW-D-R-021-tops-20230210_20230129-033504-00035E_00035N-PP-8473-v2_0_6.nc
  - https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-014-tops-20230209_20230116-153436-00034E_00037N-PP-3d4c-v2_0_6.nc
  - https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-014-tops-20230209_20230116-153411-00035E_00035N-PP-fd4d-v2_0_6.nc

In [3]:
%matplotlib inline
import os
import datetime as dt
from mintpy.utils import ptime

data_list = [
    #   sensor      mode        pass  track  frame         date1         date2         time
    # Dense fringes in the near field + snow cover --> difficult to unwrap (no unw file from ARIA too), thus, use dense offsets instead.
    # S1_A116: data acquired at 20230216 miss one frame in ESA/ASF, thus 20230228 is used instead.
    ['Sentinel-1', 'TOPS'    , 'ASC', '014', None,        '2023-01-28', '2023-02-09', '15:34'],
    ['Sentinel-1', 'TOPS'    , 'ASC', '116', None,        '2023-02-04', '2023-02-28', '15:26'],
    ['Sentinel-1', 'TOPS'    , 'DSC', '021', None,        '2023-01-29', '2023-02-10', '03:34'],

    # ALOS-2, successfully unwrap after careful masking
    # ALOS-2 local solar time: 23:40 for asc (weak iono), 11:46 for desc (strong iono)
    ['ALOS-2'    , 'ScanSAR' , 'ASC', '184', '0700_0750', '2022-09-05', '2023-02-20', '21:28'],
    ['ALOS-2'    , 'ScanSAR' , 'DSC', '077', '2850_2900', '2022-09-16', '2023-02-17', '09:33'],

    # LT1_D000_20220411_20230210:
    ['LuTan-1'   , 'Stripmap', 'DSC', '068', None,        '2022-04-11', '2023-02-10', '03:34'],

    ## Bad
    # [WIP] ALOS2_D078_20220406_20230208: not freely available from JAXA; use ARIA products directly
    # To-do: find out what corrections have been applied to ARIA
    ['ALOS-2'    , 'Stripmap', 'DSC', '078', '2830_2880', '2022-04-06', '2023-02-08', '09:40'],
    # [WIP] ALOS2_A183_20190918_20230215: low coherence due to long temporal baseline -> difficult to unwrap
    # To-do: try offset instead, need to mosaic the ALOS-2 ScanSAR [change isce2 code; low priority]
    ['ALOS-2'    , 'ScanSAR' , 'ASC', '183', '0750',      '2019-09-18', '2023-02-15', '21:21'],
]